## Stock Price Prediction Using LSTM

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
## 
data = pd.read_csv('../input/stock-price-data-for-forecasting-problems/stock_price_dada.csv')

## Importing all the libraries

In [ ]:
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import math
from IPython.display import display
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
data.head()

## Exploratory data analysis

In [ ]:
data.isnull().values.any()

In [ ]:
data.info()

In [ ]:
sns.set_style("whitegrid") 
sns.boxplot(x = 'Open', data = data) 
plt.show()

In [ ]:
sns.set_style("whitegrid") 
sns.boxplot(x = 'High', data = data,color='red') 
plt.show()

In [ ]:
sns.set_style("whitegrid") 
sns.boxplot(x = 'Low', data = data) 
plt.show()

In [ ]:
sns.set_style("whitegrid") 
sns.boxplot(x = 'Last', data = data,color='red') 
plt.show()

In [ ]:
sns.set_style("whitegrid") 
sns.boxplot(x = 'Close', data = data) 
plt.show()

In [ ]:

plt.figure(figsize = (10,5))
plt.plot(range(data.shape[0]),(data['Close']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price',fontsize=18)
plt.title("Close Price Over Time")
plt.show()

In [ ]:

plt.figure(figsize = (10,5))
plt.plot(range(data.shape[0]),(data['Open']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Open Price',fontsize=18)
plt.title("Open Price Over Time")
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(data.shape[0]),(data['High']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('High Price',fontsize=18)

plt.title("High Price Over Time")
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(data.shape[0]),(data['Low']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Low Price',fontsize=18)
plt.title("Low Price Over Time")
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(range(data.shape[0]),(data['Last']))
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Last Price',fontsize=18)
plt.title("Last Price Over Time")
plt.show()

In [ ]:
sns.set_style('whitegrid') 
sns.distplot(data['Open'], kde = False, color ='red', bins = 30) 
plt.title("Open Price Distribution")
plt.show()

In [ ]:
sns.set_style('whitegrid') 
sns.distplot(data['High'], kde = False, color ='blue', bins = 30) 
plt.title("High Price Distribution")
plt.show()

In [ ]:
sns.set_style('whitegrid') 
sns.distplot(data['Low'], kde = False, color ='green', bins = 30) 
plt.title("Low Price Distribution")
plt.show()

In [ ]:
sns.set_style('whitegrid') 
sns.distplot(data['Last'], kde = False, color ='orange', bins = 30) 
plt.title("Last Price Distribution")
plt.show()

In [ ]:
sns.set_style('whitegrid') 
sns.distplot(data['Close'], kde = False, color ='crimson', bins = 30) 
plt.title("Close Price Distribution")
plt.show()

In [ ]:

df = pd.read_csv('../input/stock-price-data-for-forecasting-problems/stock_price_dada.csv')

## Long Short Term Memory (LSTM)

In [ ]:
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:987,:]
valid = dataset[987:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))

model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
#for plotting
plt.figure(figsize=(14,5))
train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.show()

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(valid['Close'], color = 'blue', label = 'TATA Stock Price')
plt.plot(valid['Predictions'], color = 'red', label = 'Predicted TATA Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TATA Stock Price')
plt.legend()
plt.show()

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
trainScore = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: %.8f MSE (%.8f RMSE)' % (trainScore, math.sqrt(trainScore)))
